In [1]:
import os
import pandas as pd
from tqdm import tqdm
import awswrangler as wr
import math


In [3]:
def push_table(table_name,data):
    # Push 1 table, take name of the table and data to push
    path = f'{TABLE_PATH}{table_name}'

    wr.s3.to_parquet(
        df=data,
        path=path,
        database=DATABASE_NAME,
        dataset=True,
        table=table_name,
        mode='overwrite',
        index=False,
        compression=None,
    )
    
    return True

In [9]:
# def push_dataframe_chunks(df, chunk_size=100000):
#     num_chunks = math.ceil(df.shape[0] / chunk_size)
#     for i in range(num_chunks):
#         chunk_start = i * chunk_size
#         chunk_end = min((i + 1) * chunk_size, df.shape[0])
#         chunk = df.iloc[chunk_start:chunk_end]
#         table_name = f"table_chunk_{i}"  # Nom de la table pour chaque morceau
#         success = push_table(table_name, chunk)
#         if not success:
#             return False
#     return True

In [ ]:
%%time
# Chemin du dossier contenant les fichiers CSV
dossier = 'data'

# Liste pour stocker les dataframes
dataframes = []

# Variables pour le nombre total de lignes
total_lignes = 0

# Parcourir tous les fichiers du dossier avec tqdm pour afficher la progression
for fichier in tqdm(os.listdir(dossier)):
    if fichier.endswith('.csv'):
        chemin_fichier = os.path.join(dossier, fichier)
        
        # Vérifier si le fichier est vide
        if os.stat(chemin_fichier).st_size == 0:
            continue
        
        # Lire le fichier CSV dans un dataframe
        df = pd.read_csv(chemin_fichier)
        
        # Vérifier si le dataframe est vide
        if df.empty:
            continue
        
        # Vérifier si la colonne "tempsarret" est absente
        if 'tempsarret' not in df.columns:
            # Ajouter la colonne avec des cases vides
            df['tempsarret'] = ''
        
        # Stocker le dataframe dans la liste
        dataframes.append(df)
        
        # Ajouter le nombre de lignes du dataframe au total
        total_lignes += df.shape[0]

# Fusionner tous les dataframes de la liste en un seul dataframe
fusion_df = pd.concat(dataframes, ignore_index=True)
# Afficher le nombre total de lignes et la shape du dataframe final
print("Nombre total de lignes des fichiers CSV:", total_lignes)
print("Shape du dataframe final:", fusion_df.shape)


In [6]:
fusion_df

,gid,hor_theo,hor_app,etat,type,source,rs_sv_arret_p,rs_sv_cours_a,mdate,hor_real,tempsarret
0,168488116,2023-01-08T09:53:38+01:00,2023-01-08T09:53:38+01:00,NON_REALISE,REGULIER,SAEIV_TRAM,127940,2519295,2023-01-08T03:31:30+01:00,NaN,NaN
1,168488117,2023-01-08T09:54:52+01:00,2023-01-08T09:54:52+01:00,NON_REALISE,REGULIER,SAEIV_TRAM,127938,2519295,2023-01-08T03:31:30+01:00,NaN,NaN
2,168492575,2023-01-08T16:15:39+01:00,2023-01-08T16:15:39+01:00,NON_REALISE,REGULIER,SAEIV_TRAM,128069,2518974,2023-01-08T03:31:30+01:00,NaN,NaN
3,168492576,2023-01-08T16:17:06+01:00,2023-01-08T16:17:06+01:00,NON_REALISE,REGULIER,SAEIV_TRAM,128071,2518974,2023-01-08T03:31:30+01:00,NaN,NaN
4,168492577,2023-01-08T16:19:16+01:00,2023-01-08T16:19:16+01:00,NON_REALISE,REGULIER,SAEIV_TRAM,128076,2518974,2023-01-08T03:31:30+01:00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
70587949,268486129,2023-04-11T21:17:27+02:00,2023-04-11T21:17:27+02:00,REALISE,REGULIER,SAEIV_BUS,125269,2685669,2023-04-11T22:14:16+02:00,2023-04-11T21:17:37+02:00,3356.0
70587950,268486130,2023-04-11T21:18:09+02:00,2023-04-11T21:18:09+02:00,REALISE,REGULIER,SAEIV_BUS,126617,2685669,2023-04-11T22:15:10+02:00,2023-04-11T22:14:15+02:00,0.0
70587951,268486131,2023-04-11T21:19:00+02:00,2023-04-11T21:19:00+02:00,REALISE,REGULIER,SAEIV_BUS,125592,2685669,2023-04-11T22:15:38+02:00,2023-04-11T22:15:06+02:00,0.0
70587952,268486132,2023-04-11T21:21:00+02:00,2023-04-11T21:21:00+02:00,REALISE,REGULIER,SAEIV_BUS,128348,2685669,2023-04-11T22:17:39+02:00,2023-04-11T22:17:06+02:00,0.0


In [5]:
%%time

fusion_df = pd.read_pickle('fusion_df.pkl')
fusion_df

CPU times: user 22.1 s, sys: 10.2 s, total: 32.3 s
Wall time: 32.3 s


,gid,hor_theo,hor_app,etat,type,source,rs_sv_arret_p,rs_sv_cours_a,mdate,hor_real,tempsarret
0,168488116,2023-01-08T09:53:38+01:00,2023-01-08T09:53:38+01:00,NON_REALISE,REGULIER,SAEIV_TRAM,127940,2519295,2023-01-08T03:31:30+01:00,NaN,NaN
1,168488117,2023-01-08T09:54:52+01:00,2023-01-08T09:54:52+01:00,NON_REALISE,REGULIER,SAEIV_TRAM,127938,2519295,2023-01-08T03:31:30+01:00,NaN,NaN
2,168492575,2023-01-08T16:15:39+01:00,2023-01-08T16:15:39+01:00,NON_REALISE,REGULIER,SAEIV_TRAM,128069,2518974,2023-01-08T03:31:30+01:00,NaN,NaN
3,168492576,2023-01-08T16:17:06+01:00,2023-01-08T16:17:06+01:00,NON_REALISE,REGULIER,SAEIV_TRAM,128071,2518974,2023-01-08T03:31:30+01:00,NaN,NaN
4,168492577,2023-01-08T16:19:16+01:00,2023-01-08T16:19:16+01:00,NON_REALISE,REGULIER,SAEIV_TRAM,128076,2518974,2023-01-08T03:31:30+01:00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
70587949,268486129,2023-04-11T21:17:27+02:00,2023-04-11T21:17:27+02:00,REALISE,REGULIER,SAEIV_BUS,125269,2685669,2023-04-11T22:14:16+02:00,2023-04-11T21:17:37+02:00,3356.0
70587950,268486130,2023-04-11T21:18:09+02:00,2023-04-11T21:18:09+02:00,REALISE,REGULIER,SAEIV_BUS,126617,2685669,2023-04-11T22:15:10+02:00,2023-04-11T22:14:15+02:00,0.0
70587951,268486131,2023-04-11T21:19:00+02:00,2023-04-11T21:19:00+02:00,REALISE,REGULIER,SAEIV_BUS,125592,2685669,2023-04-11T22:15:38+02:00,2023-04-11T22:15:06+02:00,0.0
70587952,268486132,2023-04-11T21:21:00+02:00,2023-04-11T21:21:00+02:00,REALISE,REGULIER,SAEIV_BUS,128348,2685669,2023-04-11T22:17:39+02:00,2023-04-11T22:17:06+02:00,0.0


In [25]:
import warnings

def custom_convert_to_float(value):
    try:
        return float(value)
    except (ValueError, TypeError):
        # Gérer les cas spécifiques de conversion pour les valeurs non numériques
        if value.endswith('p7'):
            # Exemple de traitement pour la valeur '22p7'
            value = value.replace('p7', '.7')
            return float(value)
        else:
            # Autres cas de traitement ou renvoyer NaN pour les valeurs non convertibles
            return float('nan')
        
        
def push_table_by_batch(table_name,data,batch_size=1000000):
    
    # Ignorer le warning SettingWithCopyWarning
    warnings.filterwarnings("ignore", category=pd.core.common.SettingWithCopyWarning)
    
    # Push 1 table, take name of the table and data to push
    path = f'{TABLE_PATH}{table_name}'
    
    num_batches = math.ceil(len(data)/batch_size)
    total_rows_pushed = 0  # Variable pour stocker le nombre total de lignes poussées
    
    for i in tqdm(range(num_batches),desc="Processing batches"):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, len(data))
        batch = data.iloc[start_idx:end_idx]
        #print(batch)
        
        # Convertir les colonnes appropriées en type numérique
        numeric_cols = ['tempsarret','rs_sv_arret_p']  # Remplacez par les noms des colonnes à convertir
        for col in numeric_cols:
            batch[col] = batch[col].apply(custom_convert_to_float)

        #batch[numeric_cols] = batch[numeric_cols].apply(pd.to_numeric, errors='coerce')
        
        #Load table
        wr.s3.to_parquet(
            df=batch,
            path=path,
            database=DATABASE_NAME,
            dataset=True,
            table=table_name,
            mode='append',
            index=False,
            compression=None,
        )
        
        total_rows_pushed += len(batch)  # Ajouter le nombre de lignes du paquet au total

    print(f"Nombre total de lignes poussées dans la table AWS : {total_rows_pushed}")
    
    return True

In [26]:
%%time
push_table_by_batch("STG_SV_HORAI_A",fusion_df)

Processing batches: 100%|██████████| 71/71 [08:41<00:00,  7.35s/it]

Nombre total de lignes poussées dans la table AWS : 70587954
CPU times: user 7min 37s, sys: 2min 40s, total: 10min 18s
Wall time: 8min 41s


True

In [2]:
#fusion_df.to_pickle('fusion_df.pkl')

In [ ]:
#fusion_df.to_csv("STG_SV_HORAI_A.csv")

In [3]:
#Push to DSW
#push_table("STG_SV_HORAI_A",fusion_df)
# Appel de la fonction push_dataframe_chunks() pour écrire le dataframe en morceaux
# success = push_dataframe_chunks(fusion_df, chunk_size=100000)
# if success:
#     print("Dataframe fusionné écrit avec succès en morceaux.")
# else:
#     print("Erreur lors de l'écriture du dataframe fusionné.")